In [2]:
import os
import numpy as np
import pandas as pd
import torch
import os
import json
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2
import numpy as np
%matplotlib inline
from tqdm import tqdm
import glob
import pickle

In [2]:
!ls -lh

total 6.6G
-rw-rw-r-- 1 chicm_dev chicm_dev  2.2M Sep  4 11:01 detect-57-ensemble.ipynb
-rw-rw-r-- 1 chicm_dev chicm_dev  2.2M Sep  4 11:01 detect-57-ensemble-level2.ipynb
-rw-rw-r-- 1 chicm_dev chicm_dev  1.5M Sep  5 07:18 eda.ipynb
-rw-rw-r-- 1 chicm_dev chicm_dev  421M Sep  4 15:42 ensemble_detect_57_0808_1.csv
-rw-rw-r-- 1 chicm_dev chicm_dev  268M Sep  4 15:43 ensemble_detect_57_0808_1_top50.csv
-rw-rw-r-- 1 chicm_dev chicm_dev  421M Sep  4 15:43 ensemble_detect_57_0808_2_with_weights.csv
-rw-rw-r-- 1 chicm_dev chicm_dev  399M Sep  4 15:44 ensemble_detect_57_0819_1.csv
-rw-rw-r-- 1 chicm_dev chicm_dev  399M Sep  4 15:45 ensemble_detect_57_0819_2.csv
-rw-rw-r-- 1 chicm_dev chicm_dev  573M Sep  4 15:46 ensemble_detect_57_0820_1_top100.csv
-rw-rw-r-- 1 chicm_dev chicm_dev  1.2G Sep  4 15:48 ensemble_is_42_0808_1_3_models.csv
-rw-rw-r-- 1 chicm_dev chicm_dev  1.7G Sep  4 15:52 ensemble_is_42_0824.csv
-rw-rw-r-- 1 chicm_dev chicm_dev  911M Sep  4 15:53 ensemble_level2_0822.csv
-rw-rw-r

at                111606
on                 47935
holds              21726
interacts_with      9783
plays               6073
inside_of           2969
wears               2241
hits                 228
under                 82

In [3]:
''' lb34310
csv_files = [
            #'sub_sort_200_0820_2_fast_ensemble.csv',
            #'../vrd/lb23578/sub_tuple_0821_4.csv',
            #'../vrd/lb25388/sub_tuple_0825_cdiff.csv',
            #'../vrd/lb25388/sub_tuple_0825_cdiff_exclude_plays_insideof.csv',
            '../vrd/lb25388/sub_tuple_0825_cdiff_excludes_insideof_holds_plays_on.csv',
            '../vrd/insideof/sub_insideof.csv',
            #'../vrd/lb25388/sub_plays.csv',
            '../vrd/lb25388/sub_plays_0905_1.csv', # 0.03911
            '../vrd/lb25388/sub_holds_0902_1.csv', # 0.02945
            '../vrd/lb25388/sub_on_0905_1.csv', # 0.2760
            'ensemble_is_42_0808_1_3_models.csv'
            #'sub_is_42_0726_2.csv',
            ]

dfs = [pd.read_csv(x) for x in csv_files]
'''

In [ ]:
'''  lb34705
csv_files = [
            '../vrd/lb25388/sub_tuple_0825_cdiff_excludes_insideof_holds_plays_on_wears.csv',
            '../vrd/insideof/sub_insideof_0921_1.csv', # lb0.01221
            '../vrd/lb25388/sub_wears_0903_1.csv', # lb0.01093
            '../vrd/lb25388/sub_plays_0905_1.csv', # 0.03911
            '../vrd/lb25388/sub_holds_0902_1.csv', # 0.02945
            '../vrd/lb25388/sub_on_0921_1.csv', # 0.02778
            'ensemble_is_42_0808_1_3_models.csv'
            ]

dfs = [pd.read_csv(x) for x in csv_files]
'''

In [15]:
csv_files = [
            '../vrd/lb25388/sub_tuple_0825_cdiff_excludes_insideof_holds_plays_on_wears_interacts_with.csv',
            '../vrd/insideof/sub_insideof_0921_1.csv', # lb0.01221
            '../vrd/wears/sub_0921_2_1229model.csv', # lb0.01142
            '../vrd/lb25388/sub_plays_0905_1.csv', # 0.03911
            '../vrd/holds/sub_0921_1.csv', # 0.03062
            '../vrd/lb25388/sub_on_0921_1.csv', # 0.02778
            '../vrd/interacts_with/sub_interacts_with_0921_1.csv', #0.02408
            'ensemble_is_42_0808_1_3_models.csv'
            ]

dfs = [pd.read_csv(x) for x in csv_files]

In [16]:
for df in dfs:
    df.PredictionString = df.PredictionString.fillna('')

In [17]:
len(dfs[1].iloc[0].PredictionString)

0

In [18]:
# lower conf for is model, not working
# dfs[1].PredictionString = dfs[1].PredictionString.map(lambda x: get_low_conf_pred_str(x))

In [19]:
dfs[0].head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.387808 /m/04yx4 0.3756532 0.236988 0.7501363...
1,763c72626bf24534,0.77768 /m/04yx4 0.720626 0.5248438 0.950433 0...
2,7540c162180973b1,
3,c4e522dce52cb02f,0.702323 /m/04yx4 0.8541945 0.3647041 0.972777...
4,cafcd9496c808085,


In [20]:
#keep top 100
#dfs[0].PredictionString = dfs[0].PredictionString.map(lambda x: ' '.join(x.split(' ')[:1200]))

In [23]:
x = None
for d in dfs:
    if x is None:
        x = d.PredictionString
    else:
        x = x.str.cat(d.PredictionString, sep=' ')

In [24]:
dfs[0].PredictionString = x
print(len(dfs[0].iloc[0].PredictionString), len(dfs[1].iloc[0].PredictionString))

36062 0


In [25]:
dfs[0].head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.387808 /m/04yx4 0.3756532 0.236988 0.7501363...
1,763c72626bf24534,0.77768 /m/04yx4 0.720626 0.5248438 0.950433 0...
2,7540c162180973b1,0.101016 /m/04yx4 0.9498244 0.3313462 0.999...
3,c4e522dce52cb02f,0.702323 /m/04yx4 0.8541945 0.3647041 0.972777...
4,cafcd9496c808085,0.172185 /m/01_5g 0.7968998 0.0202855 0.9...


In [26]:
dfs[0].to_csv('merge_0921_plays_holds_insideof_on_wears_interacts_with.csv.zip', index=False, compression='zip')

In [27]:
!kaggle competitions submit -c open-images-2019-visual-relationship -f merge_0921_plays_holds_insideof_on_wears_interacts_with.csv.zip -m "merge plays, holds, insideof, on model"

100%|████████████████████████████████████████| 471M/471M [00:19<00:00, 24.8MB/s]
Successfully submitted to Open Images 2019 - Visual Relationship

In [15]:
!ls

detect-57-ensemble.ipynb
detect-57-ensemble-level2.ipynb
eda.ipynb
ensemble_detect_57_0808_1.csv
ensemble_detect_57_0808_1_top50.csv
ensemble_detect_57_0808_2_with_weights.csv
ensemble_detect_57_0819_1.csv
ensemble_detect_57_0819_2.csv
ensemble_detect_57_0820_1_top100.csv
ensemble_is_42_0808_1_3_models.csv
ensemble_is_42_0824.csv
ensemble_level2_0822.csv
filter_detect_submission.ipynb
filter_rel_class.ipynb
Is-classifier-6-eda.ipynb
Is-detect-42-eda.ipynb
Is-detector-result.ipynb
lb18901
lb19252
lb26760
merge_0905_1_plays_holds_insideof_on.csv
merge_df.ipynb
show-detections.ipynb
submit-detect-57.ipynb
submit-is-42-ensemble.ipynb
submit-is-42.ipynb
submit-is-6-catboost.ipynb
submit-is-6.ipynb
tmp_pos_rel.csv
top_classes_42.csv
top_classes.csv
train_lgbm.ipynb
tuple-rel-classifier.ipynb
tuple-rel-classifier-new.ipynb
tuple-rel-classifier-new-lb18901.ipynb


In [11]:
len(dfs[0].loc[dfs[0].PredictionString.isna()])

10336

In [56]:
len(dfs[0].loc[dfs[0].PredictionString.isna()])

0

In [12]:
len(dfs[1].loc[dfs[1].PredictionString.isna()])

41

In [100]:
#####
lst = ['Geeks', 'For', 'Geeks', 'is',  
            'portal', 'for', 'Geeks'] 
tmp1 = pd.DataFrame(lst, columns=['t'])
lst2 = ['a', 'b', 'c', 'd',  
            'e', 'f', 'g'] 
tmp2 = pd.DataFrame(lst2, columns=['t'])

lst3 = ['a1', 'b1', 'c', 'd',  
            'e', 'f', 'g'] 
tmp3 = pd.DataFrame(lst3, columns=['t'])
tmp3.head()

,t
0,a1
1,b1
2,c
3,d
4,e


In [4]:
df_is = pd.read_csv('ensemble_is_42_0808_1_3_models.csv')
df_is.head()

,ImageId,PredictionString
0,21135d5ed0e7a0b7,0.1689330 /m/04bcr3 0.3664692 0.7250217 0.4814...
1,763c72626bf24534,0.7671249 /m/04bcr3 0.6873273 0.8193369 0.9977...
2,7540c162180973b1,0.0219995 /m/04dr76w 0.0762036 0.3234025 0.120...
3,c4e522dce52cb02f,0.4940703 /m/04bcr3 0.8307936 0.6809913 0.9920...
4,cafcd9496c808085,0.0129044 /m/04dr76w 0.0000000 0.0600080 0.031...


In [5]:
def get_low_conf_pred_str(pred_str):
    try:
        if len(pred_str) < 1:
            return pred_str
    except:
        print('pred_str:', pred_str)
        raise
    det_items = pred_str.split(' ')
    
    res = []
    for i, c in enumerate(det_items):
        if i % 12 == 0:
            res.append(str(float(c)/4))
        else:
            res.append(c)
    return ' '.join(res)